In [ ]:
from astropy.io import fits
import astropy.constants as C
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#Reading in the data
file_prism = fits.open('NIRSPEC-PRISM.fits')
header_prism = file_prism[0].header
data_prism = file_prism[ 1 ].data
data_header_prism = file_prism[ 1 ].header

file_miri = fits.open('MIRI-LRS.fits')
header_miri = file_miri[0].header
data_miri = file_miri[ 1 ].data
data_header_miri = file_miri[ 1 ].header

In [ ]:
#I’ll just show how to convert the units for prism .
#The same idea applies to miri / lrs .
wave_prism = data_prism['WAVELENGTH'] * u.micron
flux_prism = data_prism ['FLUX '] * u.Jy #Jy
error_prism = data_prism[ 'FLUX_ERROR ']*u.Jy# Jy
#converted from Jy to ergs/s/cmˆ2/um
FLUX_prism = ( (flux_prism * C.c ) / wave_prism ** 2 ).to(u.erg /u.s/u.cm**2/u.AA).value
ERROR_prism =(( error_prism * C.c ) / wave_prism**2).to(u.erg /u.s/u.cm**2/u.AA).value
#transformed the data
FLUX_prism = FLUX_prism.astype(FLUX_prism.dtype.newbyteorder('='))
#now to trim the data
#getting rid of the nan values
trimmed_FLUX_prism = FLUX_prism [~np.isnan(FLUX_prism)]
WAVE_prism = wave_prism.value.astype(wave_prism.value.dtype.newbyteorder ('='))
#now have together they have the same size array for the wavelength array
trimmed_WAVE_prism = WAVE_prism [~np.isnan(FLUX_prism)]
#samething for the unc.error
trimmed_ERROR_prism = ERROR_prism[~np.isnan(FLUX_prism)]


#removing any negative negative
negative_indices_prism = np.where(trimmed_FLUX_prism < 0)[0]
pos_trim_FLUX_prism = np.delete(trimmed_FLUX_prism, negative_indices_prism)
pos_trim_WAVE_prism = np.delete(trimmed_WAVE_prism, negative_indices_prism)
pos_trim_ERROR_prism = np.delete(trimmed_ERROR_prism, negative_indices_prism)

In [ ]:
#Now for MIRI
wave_miri = data_miri['WAVELENGTH'] * u.micron
flux_miri = data_miri['FLUX '] * u.Jy #Jy
error_miri = data_miri[ 'FLUX_ERROR '] * u.Jy# Jy
#converted from Jy to ergs/s/cmˆ2/um
FLUX_miri = ( (flux_miri * C.c ) / wave_miri ** 2 ).to(u.erg /u.s/u.cm**2/u.AA).value
ERROR_miri =(( error_miri * C.c ) / wave_miri**2).to(u.erg /u.s/u.cm**2/u.AA).value
#transformed the data
FLUX_miri = FLUX_miri.astype(FLUX_miri.dtype.newbyteorder('='))
#now to trim the data
#getting rid of the nan values
trimmed_FLUX_miri = FLUX_miri [~np.isnan(FLUX_miri)][::-1]
WAVE_miri = wave_miri.value.astype(wave_miri.value.dtype.newbyteorder ('='))
#now have together they have the same size array for the wavelength array
trimmed_WAVE_miri = WAVE_miri[~np.isnan(FLUX_miri)][::-1]
#samething for the unc.error
trimmed_ERROR_miri = ERROR_miri[~np.isnan(FLUX_miri)][::-1]


#removing any negative values
negative_indices_miri = np.where(trimmed_FLUX_miri < 0)[0]
pos_trim_FLUX_miri = np.delete(trimmed_FLUX_miri, negative_indices_miri)
pos_trim_WAVE_miri = np.delete(trimmed_WAVE_miri, negative_indices_miri)
pos_trim_ERROR_miri = np.delete(trimmed_ERROR_miri, negative_indices_miri)

In [ ]:
plt.plot(pos_trim_WAVE_prism,pos_trim_FLUX_prism,label='NIRSpec PRISM')
plt.plot(pos_trim_WAVE_miri,pos_trim_FLUX_miri,label = 'MIRI LRS')
plt.xlabel('Wavelength [microns]')
plt.ylabel('Flux [erg/s/cm$^2$/Å]')
plt.legend();

In [ ]:
#We need to find the SNR
SNR_prism = pos_trim_FLUX_prism / pos_trim_ERROR_prism
SNR_miri = pos_trim_FLUX_miri / pos_trim_ERROR_miri

In [ ]:
plt.plot(pos_trim_WAVE_prism, SNR_prism,label='NIRSpec SNR')
plt.plot(pos_trim_WAVE_miri, SNR_miri,label = 'MIRI SNR')
plt.xlabel('Wavelength [microns]')
plt.ylabel('Signal-to-Noise')
plt.legend()
#we see that they intersect around 4.5 microns

In [ ]:
#creating a dataframe so that you can see the tables side by side.
#we need to be more accurante than just saying they intersect around 4.5 microns. But we have an idea which wavelengths we should be looking at.
#I see the SNR for MIRI becomes higher at the 11th index.
#The SNR for NIRSpec becomes lower than MIRI at the 319th index.
#This can be seen by running the next cell. 
prism_words = len(pos_trim_WAVE_prism)*['NIRSpec']
miri_words = len(pos_trim_WAVE_miri)*['MIRI']

prism_dictionary = {'Wavelength':pos_trim_WAVE_prism,
                'SNR': SNR_prism,
                'Instrument': prism_words}
df1 = pd.DataFrame(prism_dictionary)

miri_dictionary = {'Wavelength':pos_trim_WAVE_miri,
                'SNR': SNR_miri,
                'Instrument': miri_words}
df2 = pd.DataFrame(miri_dictionary)

In [ ]:
from IPython.display import display, HTML
display(HTML(f"""
    <div style="display: flex; justify-content: space-around;">
        <div>{df1.to_html()}</div>
        <div>{df2.to_html()}</div>
    </div>
"""))
#I am sure there is an easier way than to do this by eye, but that's more trouble than what its worth.

In [ ]:
#trim prism data to the 319th index since this is where the SNR becomes smaller compared to the MIRI data
final_WAVE_prism = pos_trim_WAVE_prism[:319]
final_FLUX_prism = pos_trim_FLUX_prism[:319]
final_ERROR_prism = pos_trim_ERROR_prism[:319]

#trim the miri data before the 11th index since the data before it has a smaller SNR compared to the PRISM dara
final_WAVE_miri = pos_trim_WAVE_miri[11:]
final_FLUX_miri = pos_trim_FLUX_miri[11:]
final_ERROR_miri = pos_trim_ERROR_miri[11:]

## To get the power resolution function, you will need to see my notes.

In [ ]:
NIRSpec_res = SolveNirSpecRes(final_WAVE_prism)
MIRI_res = SolveMiriRes(final_WAVE_miri)

In [ ]:
#I am multiply 10 to convert erg/s/cm^2/A to W/m^2/um
#Now, I combining both arrays
final_wave = np.concatenate( (final_WAVE_prism, final_WAVE_miri) )
final_flux = np.concatenate( (final_FLUX_prism, final_FLUX_miri) ) * 10
final_error = np.concatenate( (final_ERROR_prism, final_ERROR_miri) ) *10
final_res = np.concatenate( (NIRSpec_res, MIRI_res) )

In [ ]:
#check to see everything looks right
plt.plot(final_wave, final_flux)
plt.xlabel('Wavelength [microns]')
plt.ylabel('Flux [erg/s/cm$^2$/Å]');

In [ ]:
#now we can save it to txt file so we can run retrievals.
data = {'w':final_wave,
                  'f':final_flux,
                  'e':final_error,
                  'r':final_res}
#df = pd.DataFrame(data).to_csv('J1416B_PRISM+LRS.txt',index=False,sep='\t',header=None)